In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from scipy.ndimage import shift

Загружаем исходные данные по всем регионам.

In [2]:
all_data = pd.read_csv("joint_data.csv", index_col=0, parse_dates=True)

In [3]:
all_data.head()

,2068,2118,2168,2069,2119,1221,1172,1222,1272,1173,...,1437,1338,1388,1438,1339,1389,1439,1390,1441,1442
2013-01-01 00:00:00,19.0,68.0,67.0,6.0,18.0,34.0,19.0,56.0,73.0,63.0,...,66.0,119.0,68.0,28.0,28.0,44.0,17.0,55.0,20.0,36.0
2013-01-01 01:00:00,10.0,81.0,15.0,3.0,1.0,96.0,49.0,120.0,132.0,104.0,...,106.0,128.0,93.0,55.0,39.0,77.0,54.0,100.0,43.0,58.0
2013-01-01 02:00:00,2.0,28.0,16.0,0.0,0.0,81.0,51.0,93.0,110.0,124.0,...,89.0,89.0,104.0,65.0,37.0,82.0,42.0,94.0,49.0,65.0
2013-01-01 03:00:00,2.0,6.0,1.0,0.0,0.0,91.0,39.0,86.0,78.0,111.0,...,76.0,48.0,80.0,43.0,33.0,58.0,45.0,70.0,41.0,41.0
2013-01-01 04:00:00,3.0,30.0,16.0,0.0,2.0,48.0,24.0,42.0,42.0,69.0,...,78.0,37.0,94.0,30.0,38.0,44.0,18.0,47.0,25.0,30.0


Определимся теперь с набором потенциальных признаков. 

Я не буду использовать прогнозы ARIMA как регрессионные признаки, несмотря на то, что в задании их использование рекомендуется - хотя и не обязательно. Дело в том, что подогнанные значения модели и её прогнозы (которые нужны для построения прогноза общей регрессионной модели) суть разные величины, и использовать их как один и тот же признак, строго говоря, некорректно.

Для учёта связности спроса между регионами, в качестве дополнительных признаков я возьму лаги по всем имеющимся регионам.

Дополнительно я возьму в качестве признаков характерные шаблоны, вычисленные как среднее на некотором окне (скажем, неделя) - чтобы учесть периодичность, которая не выявляется гармоническими признаками.

Для стабилизации модели воспользуемся $L^2$ - регуляризатором. Поскольку для каждого региона требуется настраивать шесть моделей и делать прогноз с использованием всех шести, попробуем для удобства завернуть весь процесс в класс.

In [4]:
class multipleRegressionPredictor:
    def __init__(self, all_y, x):
        y = all_y.copy()
        self.__means = y.mean()
        self.__sd = y.std()
        self.__y = (y - self.__means) / self.__sd
        self.__x = x
        self.__models = dict()
        for column in self.__y.columns:
            self.__models[column] = dict()      
    
    
    def generateHarmonicRegressors(self, k_week=(1, 10), k_year=(30, 40), crossprod=False):
        # Clearing all available models
        self.__models = dict()
        for column in self.__y.columns:
            self.__models[column] = dict() 
        # Constructing regressors
        week_values = np.arange(k_week[0], k_week[1] + 1)
        year_values = np.arange(k_year[0], k_year[1] + 1)
        week_sin = reduce(lambda x, y: np.hstack((x, y)),
            map(lambda k: np.sin(np.pi * self.__x * k / 84), week_values))
        week_cos = reduce(lambda x, y: np.hstack((x, y)),
            map(lambda k: np.cos(np.pi * self.__x * k / 84), week_values))
        year_sin = reduce(lambda x, y: np.hstack((x, y)),
            map(lambda k: np.sin(np.pi * self.__x * k / 4383), year_values))
        year_cos = reduce(lambda x, y: np.hstack((x, y)),
            map(lambda k: np.cos(np.pi * self.__x * k / 4383), year_values))
        harmonic_part = np.hstack((week_sin, week_cos, year_sin, year_cos))
        if crossprod:
            n_harmonics = (k_week[1] - k_week[0] + 1) * 2 + (k_year[1] - k_year[0] + 1) * 2
            n_crossprods = (n_harmonics + 1) * n_harmonics // 2
            cross_harmonics = np.zeros((harmonic_part.shape[0], n_crossprods))
            indices = np.arange(n_harmonics)
            offset = 0
            for i in indices:
                cross_harmonics[:, offset:(offset + n_harmonics - i)] = (harmonic_part[:, i:] * 
                                                                         harmonic_part[:, i][:, np.newaxis])
                offset += n_harmonics - i
            harmonic_part = np.hstack((harmonic_part, cross_harmonics))
        
        self.__full_X = np.hstack((self.__x, harmonic_part))
        self.__full_X -= np.mean(self.__full_X, axis=0)
        self.__full_X /= np.std(self.__full_X, axis=0, ddof=1)
    
    
    def expandPeriodicStructure(self, window, indices):
        periodic = np.zeros(self.__y.shape)
        for i in np.arange(self.__y.shape[1]):
            # Generating periodic structure
            col = self.__y.iloc[:, i].values
            col_to_period = col[indices]
            n_points = col_to_period.shape[0]
            N = n_points // window
            per = np.mean(a=np.reshape(a=col_to_period[:(N * window)], newshape=(N, window)).transpose(), axis=1)
            # Expanding periodic structure
            periodic[:, i] = np.tile(A=per, reps=(1 + periodic.shape[0] // window))[:periodic.shape[0]]
        periodic -= np.mean(periodic)
        periodic /= np.std(periodic, ddof=1)
        self.__full_X = np.hstack((self.__full_X, periodic))
    
    
    def getFullX(self):
        return self.__full_X[:, :]
    
    
    def __augmentHarmonicsWithLags(self, target_column, indices, lags):
        n_points = np.sum(indices)
        n_lags = 0
        for lag in lags:
            n_lags += 1
            new_indices = shift(input=indices, cval=False, shift=-lag)
            new_n_points = np.sum(new_indices)
            if new_n_points < n_points:
                n_points = new_n_points
        n_col = self.__y.shape[1]
        lagged_x = np.zeros((n_points, n_col * n_lags))
        offset = 0
        for lag in lags:
            lagged_x[:, offset:(offset + n_col)] = self.__y.values[shift(input=indices, cval=False, shift=-lag), 
                                                                   :][-n_points:, :]
            offset += n_col
        return (self.__y.loc[indices, target_column].values[-n_points:], np.hstack((self.__full_X[-n_points:, :], lagged_x)))
        
      
    def fitARModel(self, target_column, train_indices, alpha, model_shift, model_lags):
        model_lags = np.asarray(model_lags)
        model_lags += model_shift - 1
        y, augmented_X = self.__augmentHarmonicsWithLags(target_column, train_indices, model_lags)
        lm = Ridge(alpha=alpha, fit_intercept=False)
        lm.fit(augmented_X, y)
        self.__models[target_column][model_shift] = {"model": lm, "lags": model_lags}

        
    def predict(self, target_column, test_indices, align_forecast=True):
        predictions = []
        shifts = []
        for model_shift in self.__models[target_column]:
            model = self.__models[target_column][model_shift]["model"]
            model_lags = np.copy(self.__models[target_column][model_shift]["lags"])
            if not align_forecast:
                model_lags -= model_shift - 1
            y, augmented_X = self.__augmentHarmonicsWithLags(target_column, test_indices, model_lags)
            prediction = ((model.predict(augmented_X)[:, np.newaxis] * self.__sd.loc[target_column] + 
                           self.__means.loc[target_column]))
            predictions.append(prediction)
            shifts.append(model_shift)
        return np.hstack(predictions)[:, np.argsort(shifts)]

Теперь создадим объект нашего класса

In [5]:
x = (all_data.index - all_data.index[0]).map(lambda x: x.total_seconds() / 3600)
mpr = multipleRegressionPredictor(all_y=all_data, x=x.values[:, np.newaxis])

Зададим положения обучающей и тестовой выборок

In [6]:
train_indices = all_data.index < pd.to_datetime("2016-05-01")
test_indices = np.logical_and(all_data.index >= pd.to_datetime("2016-05-01"), all_data.index < pd.to_datetime("2016-06-01"))

Сгенерируем гармонические признаки

In [7]:
mpr.generateHarmonicRegressors(k_week=(1, 10), k_year=(1, 60), crossprod=False)

Сгенерируем негармонические признаки

In [8]:
mpr.expandPeriodicStructure(indices=train_indices, window=168)

Зададим авторегрессионные лаги.

In [9]:
AR_lags=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
         13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
         48, 72, 96, 120, 144, 168]

Зададим набор параметров регуляризации.

In [10]:
alphas = np.asarray(list(map(lambda x: np.power(x, 10), np.arange(9))))

Подберём модели для прогнозирования на 1-6 часов вперёд для каждого региона. Параметр регуляризации подбирается индивидуально для каждой модели.

In [11]:
best_alphas = np.zeros((all_data.shape[1], 6))
Q = 0

In [12]:
%%time
for j, region in enumerate(all_data.columns):
    test_series = all_data.loc[test_indices, region]
    all_errors = np.zeros((len(alphas), 6))
    # Scanning for best alpha
    for k, alpha in enumerate(alphas):
        for i in range(6):
            mpr.fitARModel(target_column=region,
                           train_indices=train_indices,
                           model_shift=i + 1,
                           alpha=alpha,
                           model_lags=AR_lags)
        full_prediction = mpr.predict(target_column=region,
                                      test_indices=test_indices,
                                      align_forecast=True)
        difference = full_prediction - test_series.values[:, np.newaxis]
        all_errors[k, :] = np.mean(a=np.abs(difference), axis=0)
    min_alphas_indices = np.argsort(a=all_errors, axis=0)[0, :]
    current_min_alphas = alphas[min_alphas_indices]
    # Setting best alphas
    best_alphas[j, :] = current_min_alphas
    for i, alpha in enumerate(current_min_alphas):
        mpr.fitARModel(target_column=region,
                       train_indices=train_indices,
                       model_shift=i + 1,
                       alpha=alpha,
                       model_lags=AR_lags)
    # Calculating prediction
    full_prediction = mpr.predict(target_column=region,
                                  test_indices=test_indices,
                                  align_forecast=False)
    # Adding terms to total error
    lQ = 0
    for i in range(739):
        d = np.sum(np.abs(full_prediction[i, :] - test_series[i:(i + 6)]))
        lQ += d
        Q += d
    # Updating output
    print "region {0} processed with Q {1}, overall Q is {2}".format(region, lQ / (739 * 6), Q / (739 * 6 * (j + 1)))

region 2068 processed with Q 24.4122000762, overall Q is 24.4122000762
region 2118 processed with Q 30.0275442643, overall Q is 27.2198721702
region 2168 processed with Q 19.6845048004, overall Q is 24.7080830469
region 2069 processed with Q 4.6606668812, overall Q is 19.6962290055
region 2119 processed with Q 15.3608891157, overall Q is 18.8291610276
region 1221 processed with Q 2.72390022046, overall Q is 16.144950893
region 1172 processed with Q 3.05827698062, overall Q is 14.2754260484
region 1222 processed with Q 4.21433694957, overall Q is 13.0177899111
region 1272 processed with Q 3.27386941194, overall Q is 11.9351320778
region 1173 processed with Q 7.16025265075, overall Q is 11.4576441351
region 1223 processed with Q 5.49381653052, overall Q is 10.9154779892
region 1273 processed with Q 5.49855156363, overall Q is 10.4640674538
region 1174 processed with Q 7.16997390952, overall Q is 10.2106756427
region 1224 processed with Q 6.28914562901, overall Q is 9.93056635599
region 1